# Infer Ideology Labels

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
base_dir = os.getenv('BASEDIR')
from pathlib import Path
import re
import pickle as pk
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
from flaml.default import LGBMClassifier
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
base_dir = os.getenv('BASEDIR')
from pathlib import Path
import re
import pickle as pk
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
# from lightgbm import LGBMClassifier
from flaml.default import LGBMClassifier

from tqdm import tqdm

In [ ]:
datasets = ['qanda', 'ausvotes','socialsense','riot', 'parler']
datasets_twitter = {'qanda', 'ausvotes', 'riot'}

In [ ]:
gt_fr_name = 'URLb_FR'
for dataset in tqdm(datasets):
    if dataset in datasets_twitter:
        features_lr=['use', 'rt']
        features_fr=['use', 'ht']
        # features_fr= ['use']
    else:
        features_lr= ['use']
        features_fr= ['use']
    
    granularity='_per_user'
    gt='URL_LR'
    data_path = os.path.join(base_dir,'data','01_raw_data',dataset, dataset+granularity+'.pk')
    with open(data_path, 'rb') as rf:
        data = pk.load(rf).reset_index(drop=False)
        
    def load_vector(fp):
        with open(fp, 'rb') as rf:
            vec = pk.load(rf)
        return vec
    
    feature_paths_lr = [os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk') for feature in features_lr]
    features_lr = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths_lr]))
    
    feature_paths_fr = [os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk') for feature in features_fr]
    features_fr = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths_fr]))
    
    gt_stance = load_vector(os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_URL_LR_'+granularity+'.pk'))
    gt_fr = load_vector(os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+gt_fr_name+'_'+granularity+'.pk'))    

    # Left-Right
    et = LGBMClassifier(is_unbalance=True,n_estimators=200, n_jobs=-1, verbose=-1, seed=41)
    mask_stance = gt_stance >=0
    y_stance = gt_stance[mask_stance]
    X_stance = features_lr[mask_stance]
    print('Training:', 'lr')
    et.fit(X_stance,y_stance)
    print('Trained:', 'lr')
    predicted_stance = et.predict(features_lr)
    predicted_stance_proba = et.predict_proba(features_lr)
    
    # Far-Right
    et = LGBMClassifier(is_unbalance=True, n_jobs=-1, verbose=-1, seed=123)
    mask_fr = gt_fr >=0
    y_fr = gt_fr[mask_fr]
    X_fr = features_fr[mask_fr]
    print('Training:', 'fr')
    et.fit(X_fr,y_fr)
    print('Trained:', 'fr')
    predicted_fr = et.predict(features_fr)
    predicted_fr_proba = et.predict_proba(features_fr)
                        
    pred_stance = pd.Series([['N', 'L', 'R'][i] for i in predicted_stance]).values
    
    label = pred_stance
    label[predicted_fr] = 'FR'
    data['label'] = label
    data['lr_prob_left'] = predicted_stance_proba[:,1]

    data['lr_prob_right'] = predicted_stance_proba[:,2]

    data['lr_label'] = pd.Series([['N', 'L', 'R'][i] for i in predicted_stance]).values

    data['fr_prob'] = predicted_fr_proba[:,1]

    data['fr_label'] = predicted_fr

    Path(os.path.join(base_dir,'data','04_results','labels')).mkdir( parents=True, exist_ok=True )
    data[['uid','label','lr_prob_left','lr_prob_right', 'lr_label', 'fr_prob', 'fr_label']].to_csv(os.path.join(base_dir,'data','04_results','labels',dataset+'_extended.csv'))

In [ ]:
data['lr_prob_left'] = predicted_stance_proba[:,1]

data['lr_prob_right'] = predicted_stance_proba[:,2]

data['lr_label'] = pred_stance

data['fr_prob'] = predicted_fr_proba[:,1]

data['fr_label'] = predicted_fr

Path(os.path.join(base_dir,'data','04_results','labels')).mkdir( parents=True, exist_ok=True )
data[['uid','label','lr_prob_left','lr_prob_right', 'lr_label', 'fr_prob', 'fr_label']].to_csv(os.path.join(base_dir,'data','04_results','labels',dataset+'_extended.csv'))